In [ ]:
%matplotlib inline
import cv2
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import numpy as np
from numpy.lib.stride_tricks import as_strided
import PIL
from pandas import DataFrame
import pandas as pd
from IPython.display import display, HTML
from skimage.exposure import rescale_intensity
import json
import os
from itertools import product
import itertools
import random
from os import listdir

from utils import *

pd.options.display.html.border = 0
pd.options.display.float_format = '{:,.2f}'.format

In [ ]:
data_path = "./data/"
cars = [cv2.imread(data_path+img, 1) for img in listdir(data_path)]
imshow(cars[0])

In [ ]:
def median_filter(imgs):
    cars_stacked = np.stack(cars,axis=2)
    cars_gray = [cv2.cvtColor(car, cv2.COLOR_BGR2GRAY) for car in imgs]
    cars_gray_stacked = np.stack(cars_gray,axis=2)
    cars_median_arg = np.argsort(cars_gray_stacked,axis=2)[:,:,1]

    m, n, _ = cars[0].shape
    i, j = np.indices((m, n))
    cars_color = cars_stacked[i, j, cars_median_arg[i, j]]
    
    return cars_color
        

In [ ]:
imshow(median_filter(cars))

The result of the median filter is very good, however there's some car artifacts on the right where, there are cars in 2 pictures.